In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.linear_model import SGDRegressor


In [54]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [55]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [56]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

## PREPROCESSING

In [57]:
# df_train.columns[df_train.isnull().any()]

In [58]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [59]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [60]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [61]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [62]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [63]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [64]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [65]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [66]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [67]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [68]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [69]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [70]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [71]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [72]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [73]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

Deleting Outliers.

Here we are deleting outliers using Inter Quartile Range Strategy. Since for a column ouliers are in a specifc row and for different columns the oulier contained rows are different and if we delete those specific row then that row in output label should also be deleted thats why we are first adding label into our features after that we will separate the label.

In [74]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here

In [75]:
# Outlier delete

for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)] 
    df_train = df_train[(df_train[col] >=l)] 

[ -55.0 , 145.0 ]
[ 30.0 , 110.0 ]
[ 2440.0 , 16936.0 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1909.0 , 2061.0 ]
[ -255.75 , 426.25 ]
[ -1060.5 , 1767.5 ]
[ 0.0 , 0.0 ]
[ -579.875 , 1725.125 ]
[ 199.0 , 2037.0 ]
[ -1092.375 , 1820.625 ]
[ 0.0 , 0.0 ]
[ -1.5 , 2.5 ]
[ 0.0 , 0.0 ]
[ -0.5 , 3.5 ]
[ -1.5 , 2.5 ]
[ 0.5 , 4.5 ]
[ 1.0 , 1.0 ]
[ 2.0 , 10.0 ]
[ -1.5 , 2.5 ]
[ 1903.0 , 2063.0 ]
[ -19.0 , 933.0 ]
[ -239.625 , 399.375 ]
[ -97.5 , 162.5 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.5 , 12.5 ]
[ 2004.0 , 2012.0 ]


In [76]:

output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

Here we are encoding our categorical columns using 'one_hot_encoding' as a strategy. We will be experimenting other strategies later.

In [77]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [78]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [79]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,0,0,0,1,0,0,0,0,1,0
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,0,0,0,1,0,0,0,0,1,0
6,20,75.0,10084,8,5,2005,186.0,1369,0,317,...,0,0,0,1,0,0,0,0,1,0
10,20,70.0,11200,5,5,1965,0.0,906,0,134,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,63.0,8500,7,5,2004,106.0,0,0,1422,...,0,0,0,1,0,0,0,0,1,0
1448,50,70.0,11767,4,7,2000,0.0,0,0,560,...,0,0,0,1,0,0,0,0,1,0
1451,20,78.0,9262,8,5,2009,194.0,0,0,1573,...,0,1,0,0,0,0,0,0,0,1
1454,20,62.0,7500,7,5,2005,0.0,410,0,811,...,0,0,0,1,0,0,0,0,1,0


In [80]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>0:
        print(col)

Now we will be scaling our columns using Min Max Scaler which will make our value in the range 0 to 1.Scaling is done due to following resons :
1. To make the gradient descent converge fast so to reduce the error as much as possible
2. To make all features on a specific scale.

We do scaling mainly for the distance based algorithms such as KNN,K-Means , SVM and so on. Since it helps them to reduce the error using radient very quickly. But for tree based algorithms we didnt need sacling since their accuracy is based on the split so even though the values are scaled the splitting won't be affected.And since we are using both types of algorithms we will us


In [81]:
scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),index=df_train.index, columns=df_train.columns)
df_train


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.4,0.4375,0.400706,0.625,0.333333,0.883333,0.463357,0.436072,0.0,0.087771,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.4,0.4750,0.602391,0.625,0.333333,0.866667,0.382979,0.300185,0.0,0.253950,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.4,0.6750,0.819203,0.750,0.333333,0.833333,0.827423,0.404571,0.0,0.286717,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.5625,0.518404,0.750,0.333333,0.916667,0.439716,0.845584,0.0,0.185489,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
10,0.0,0.5000,0.598790,0.375,0.333333,0.250000,0.000000,0.559605,0.0,0.078408,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,0.0,0.4125,0.404307,0.625,0.333333,0.900000,0.250591,0.000000,0.0,0.832066,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1448,0.3,0.5000,0.639631,0.250,1.000000,0.833333,0.000000,0.000000,0.0,0.327677,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1451,0.0,0.6000,0.459195,0.750,0.333333,0.983333,0.458629,0.000000,0.0,0.920421,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1454,0.0,0.4000,0.332277,0.625,0.333333,0.916667,0.000000,0.253243,0.0,0.474547,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [82]:
from sklearn.metrics import r2_score

In [83]:
X=df_train
Y=output_col

In [84]:
avg_accuracies={}
highest_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    avg_accuracies[al]=0
    highest_accuracies[al]=0

In [85]:
def algo_testing(algo:str,test_size:float,X,Y,evaluation_metric:str,random_state,shuffle):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=random_state,shuffle=shuffle)
   
    model.fit(X_train,Y_train)
    predicted=model.predict(X_test)
    
    r2=evaluation_metric(Y_test, predicted)
    print(f"Testing r2_score of {model} is ",r2)
    print(f"Training r2_score of {model} is ", model.score(X_train,Y_train))
    if highest_accuracies[algo]<r2:
        
        highest_accuracies[algo]=r2
    avg_accuracies[algo]+=r2


In [253]:

algo_testing(algo="LinearRegression()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)



Testing r2_score of LinearRegression() is  -1.276364163299197e+20
Training r2_score of LinearRegression() is  0.9612309876009716


In [254]:

for i in range(50):
    algo_testing(algo="RandomForestRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["RandomForestRegressor()"]/=50

for i in range(50):

    algo_testing(algo="SVR()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SVR()"]/=50

for i in range(50):

    algo_testing(algo="KNeighborsRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["KNeighborsRegressor()"]/=50

for i in range(50):

    algo_testing(algo="SGDRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SGDRegressor()"]/=50

for i in range(50):

    algo_testing(algo="DecisionTreeRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["DecisionTreeRegressor()"]/=50

for i in range(50):

    algo_testing(algo="ElasticNet()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["ElasticNet()"]/=50

for i in range(50):

    algo_testing(algo="Lasso()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Lasso()"]/=50

for i in range(50):

    algo_testing(algo="Ridge()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Ridge()"]/=50

Testing r2_score of RandomForestRegressor() is  0.8723512361903724
Training r2_score of RandomForestRegressor() is  0.9823870365359836
Testing r2_score of RandomForestRegressor() is  0.8017741645407369
Training r2_score of RandomForestRegressor() is  0.9828130109326186
Testing r2_score of RandomForestRegressor() is  0.9024726184234038
Training r2_score of RandomForestRegressor() is  0.9810277386999587
Testing r2_score of RandomForestRegressor() is  0.9134730775488767
Training r2_score of RandomForestRegressor() is  0.9799583146836488
Testing r2_score of RandomForestRegressor() is  0.8233234813884853
Training r2_score of RandomForestRegressor() is  0.9831212198346928
Testing r2_score of RandomForestRegressor() is  0.9020723364458312
Training r2_score of RandomForestRegressor() is  0.9796442453964255
Testing r2_score of RandomForestRegressor() is  0.913351999811681
Training r2_score of RandomForestRegressor() is  0.9799166993578784
Testing r2_score of RandomForestRegressor() is  0.901521

Training r2_score of SVR() is  -0.011219435659691301
Testing r2_score of SVR() is  -0.010912832945905837
Training r2_score of SVR() is  -0.007834561695249542
Testing r2_score of SVR() is  -0.02921450490940347
Training r2_score of SVR() is  -0.006139658494770606
Testing r2_score of SVR() is  -0.007144598389901224
Training r2_score of SVR() is  -0.0038005581338509398
Testing r2_score of SVR() is  -0.011519736848678575
Training r2_score of SVR() is  -0.00994501856000607
Testing r2_score of SVR() is  -0.05558505363626676
Training r2_score of SVR() is  -0.008562739507979922
Testing r2_score of SVR() is  -0.007225526903160917
Training r2_score of SVR() is  -0.008793135872451341
Testing r2_score of SVR() is  -0.003580948654168692
Training r2_score of SVR() is  -0.003498022344623708
Testing r2_score of SVR() is  0.0004752288436373764
Training r2_score of SVR() is  -0.011707808632968053
Testing r2_score of SVR() is  -0.0013469541212567382
Training r2_score of SVR() is  -0.007507038108945485
Tes

Testing r2_score of KNeighborsRegressor() is  0.7195008935057937
Training r2_score of KNeighborsRegressor() is  0.8450616754654936
Testing r2_score of KNeighborsRegressor() is  0.7700573427059347
Training r2_score of KNeighborsRegressor() is  0.8385529667843157
Testing r2_score of KNeighborsRegressor() is  0.7046363394174646
Training r2_score of KNeighborsRegressor() is  0.8437045174089037
Testing r2_score of KNeighborsRegressor() is  0.7654971767549521
Training r2_score of KNeighborsRegressor() is  0.8351615697489537
Testing r2_score of KNeighborsRegressor() is  0.7855670130802398
Training r2_score of KNeighborsRegressor() is  0.8321787636826019
Testing r2_score of KNeighborsRegressor() is  0.7610005769763277
Training r2_score of KNeighborsRegressor() is  0.8367421292434419
Testing r2_score of KNeighborsRegressor() is  0.7133256048733277
Training r2_score of KNeighborsRegressor() is  0.846634153297998
Testing r2_score of KNeighborsRegressor() is  0.7245899301887799
Training r2_score o

Testing r2_score of DecisionTreeRegressor() is  0.7238751109492434
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7347536858515633
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6855615677557921
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.672097793383094
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6458789510230101
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6360604458438568
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6944816533130262
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7523542569898649
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor

Testing r2_score of ElasticNet() is  0.7282663141962318
Training r2_score of ElasticNet() is  0.7305761214630229
Testing r2_score of ElasticNet() is  0.6651360860144246
Training r2_score of ElasticNet() is  0.7324406867080012
Testing r2_score of ElasticNet() is  0.7240612233193742
Training r2_score of ElasticNet() is  0.7294071439897956
Testing r2_score of ElasticNet() is  0.721226621419195
Training r2_score of ElasticNet() is  0.7361984613326182
Testing r2_score of ElasticNet() is  0.6730507525514462
Training r2_score of ElasticNet() is  0.7377773717662832
Testing r2_score of ElasticNet() is  0.7537054657601047
Training r2_score of ElasticNet() is  0.7333561573431979
Testing r2_score of ElasticNet() is  0.7171003873318778
Training r2_score of ElasticNet() is  0.7412750099427359
Testing r2_score of ElasticNet() is  0.7452119560201793
Training r2_score of ElasticNet() is  0.7239823350908269
Testing r2_score of ElasticNet() is  0.6904246057028334
Training r2_score of ElasticNet() is  0.7

...

Testing r2_score of Lasso() is  0.8781598247999758
Training r2_score of Lasso() is  0.9668621832797809
Testing r2_score of Lasso() is  0.8906632376490389
Training r2_score of Lasso() is  0.9585914449054536


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.437e+09, tolerance: 1.689e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.406e+09, tolerance: 1.855e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scal

Testing r2_score of Lasso() is  0.8962127542223992
Training r2_score of Lasso() is  0.9617005569734052
Testing r2_score of Lasso() is  0.8813076904314352
Training r2_score of Lasso() is  0.9638358981251602
Testing r2_score of Lasso() is  0.9054381552616075
Training r2_score of Lasso() is  0.957577975512343


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.145e+09, tolerance: 1.717e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.816e+09, tolerance: 1.811e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.9077353789228094
Training r2_score of Lasso() is  0.9604394430941324
Testing r2_score of Lasso() is  0.91826032636002
Training r2_score of Lasso() is  0.9587756214156911
Testing r2_score of Ridge() is  0.8916858943658152
Training r2_score of Ridge() is  0.956819190823306
Testing r2_score of Ridge() is  0.8972578391883489
Training r2_score of Ridge() is  0.956583527272496
Testing r2_score of Ridge() is  0.9046345388635012
Training r2_score of Ridge() is  0.9584815931764888
Testing r2_score of Ridge() is  0.9111884570396261
Training r2_score of Ridge() is  0.9529710653137745
Testing r2_score of Ridge() is  0.8970065711990101
Training r2_score of Ridge() is  0.9598250593416315
Testing r2_score of Ridge() is  0.9118506434742929
Training r2_score of Ridge() is  0.9569031271340488
Testing r2_score of Ridge() is  0.8985774726656613
Training r2_score of Ridge() is  0.9547552220323933
Testing r2_score of Ridge() is  0.9006248619102664
Training r2_score of Ridge

In [255]:
avg_accuracies

{'LinearRegression()': -8.846582443673856e+22,
 'RandomForestRegressor()': 0.8708935281240516,
 'SVR()': -0.017696767888052092,
 'KNeighborsRegressor()': 0.7412159965951338,
 'SGDRegressor()': 0.8944786944513179,
 'DecisionTreeRegressor()': 0.7029496360239272,
 'ElasticNet()': 0.7235271663380395,
 'Lasso()': 0.8931990189896344,
 'Ridge()': 0.8977728988900817}

In [256]:
highest_accuracies

{'LinearRegression()': 0.8836007422648474,
 'RandomForestRegressor()': 0.9218255057958525,
 'SVR()': 0.0006368230735411817,
 'KNeighborsRegressor()': 0.8149531717130634,
 'SGDRegressor()': 0.9284783900278877,
 'DecisionTreeRegressor()': 0.8345231795265835,
 'ElasticNet()': 0.7983064401678929,
 'Lasso()': 0.9378178082075785,
 'Ridge()': 0.9360173271545263}